In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

df_meta = pd.read_csv('../processed_data/clean_data/metadata.csv', index_col = 0)


In [2]:
tissueid2tissue = {ti:t for ti, t in zip(df_meta.tissue_id, df_meta.tissue)}
subject2clinical = {v[0]:v[1] for v in df_meta[['subject_id', 'clinical']].values}
pid2subject = {v[0]:v[1] for v in df_meta[['pid', 'subject_id']].values}
    

In [3]:
for fd in ['hallmark', 'kegg']:
    fn_in = './%s/score.csv' % fd
    fn_out = './%s/score_mean_tissue.csv' % fd

    df_score = pd.read_csv(fn_in).T
    id2pathway = list(df_score.columns)

    df_score['celltype'] = df_meta.loc[df_score.index, 'celltype']
    df_score['pid'] = df_meta.loc[df_score.index, 'pid']
    df_score['subject_id'] = df_meta.loc[df_score.index, 'subject_id']
    df_score['clinical'] = df_meta.loc[df_score.index, 'clinical']
    df_score['tissue'] = df_meta.loc[df_score.index, 'tissue']
    df_score['tissue_id'] = df_meta.loc[df_score.index, 'tissue_id']

    df_mean = df_score.groupby(['pid', 'celltype', 'tissue_id']).mean()
    index_mat = df_mean.index.values
    df_mean = df_mean.fillna(0)

    df_mean['subject_id'] = [pid2subject[i[0]] for i in index_mat]
    df_mean['clinical'] = [subject2clinical[i] for i in df_mean.subject_id]
    df_mean['tissue'] = [tissueid2tissue[i[2]] for i in index_mat]
    df_mean['covid'] = (df_mean.clinical != 'Ctrl').astype('int')

    df_mean = df_mean[['subject_id', 'tissue', 'clinical', 'covid'] + id2pathway]

    df_mean.to_csv(fn_out)
